# Covariance matrix

The first thing to do to calculate the DSS is to compute the covariance matrices of our unbiased signal and of our filtered signal. The covariance matrix will synthesize the variances of each electrode (the diagonal corresponds to the variance of one electrode).

The formula of the covariance is the following:

$$cov(X,Y) = \sum_{i=1}^{N}\frac{(x_i-\bar{x})(y_i-\bar{y})}{N}$$

So it is the difference from the mean of the variable $\boldsymbol{y}$ weighted by the difference from the mean of the variable $\boldsymbol{y}$ added for each data point.

#### Example 1.

These variables can be in a matrix.

$
\boldsymbol{A}=
\begin{bmatrix}
    1 & 3 & 5\\\\
    5 & 4 & 1\\\\
    3 & 8 & 6
\end{bmatrix}
$

We will calculate the covariance of each column with each other. Let's start with column 1 and column 2:

$\boldsymbol{x} = \begin{bmatrix}
    1\\\\
    5\\\\
    3
\end{bmatrix}
$ and $\boldsymbol{y} = \begin{bmatrix}
    3\\\\
    4\\\\
    8
\end{bmatrix}$. So $\boldsymbol{\bar{x}}=3$, $\boldsymbol{\bar{y}}=5$ and $N=3$.

$cov(X,Y) = \frac{(1-3)(3-5)+(5-3)(4-5)+(3-3)(8-5)}{3}=\frac{-6}{3}=-2$